# Backtesting Data Trader's Stochastic + RSI + MACD trading strategy 
### here is the link to his video : https://www.youtube.com/watch?v=hh3BKTFE1dc

In [3]:
!pip install ta

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
Using legacy 'setup.py install' for ta, since package 'wheel' is not installed.
    Running setup.py install for ta: started
    Running setup.py install for ta: finished with status 'done'


In [1]:
!pip install yfinance

In [4]:

import yfinance as yf
import numpy as np 
import ta
import pandas as pd

In [5]:
df = yf.download('EURUSD=X', start='2021-09-15', interval='30m') # going back 60 days

[*********************100%***********************]  1 of 1 completed


In [6]:
# Now I will calculate the technical indicators 
df['%K'] = ta.momentum.stoch(df.High, df.Low, df.Close, window=14, smooth_window=3)

In [7]:
df['%D'] = df['%K'].rolling(3).mean()

In [8]:
df['rsi'] = ta.momentum.rsi(df.Close, window=14)

In [9]:
df['macd']= ta.trend.macd_diff(df.Close) 

In [10]:
df.head()

,Open,High,Low,Close,Adj Close,Volume,%K,%D,rsi,macd
Datetime,,,,,,,,,,
2021-09-15 00:00:00+01:00,1.180916,1.180916,1.180359,1.180498,1.180498,0,NaN,NaN,NaN,NaN
2021-09-15 00:30:00+01:00,1.180916,1.181056,1.180498,1.181056,1.181056,0,NaN,NaN,NaN,NaN
2021-09-15 01:00:00+01:00,1.180638,1.180916,1.180498,1.180638,1.180638,0,NaN,NaN,NaN,NaN
2021-09-15 01:30:00+01:00,1.180777,1.181056,1.180498,1.180916,1.180916,0,NaN,NaN,NaN,NaN
2021-09-15 02:00:00+01:00,1.180916,1.181474,1.180916,1.181195,1.181195,0,NaN,NaN,NaN,NaN


In [11]:
#drop NaN
df.dropna(inplace=True)

In [13]:
df2 = pd.DataFrame()
for i in range(1,4): # this is 3 lags
    mask = (df['%K'].shift(i) < 20) & (df['%D'].shift(i) < 20)
    df2 = df2.append(mask, ignore_index=True)

In [14]:
# this is the buy signal, it chekcs if there is a cross within the last 3 days, so if the vertical sum is above 0 we have a buying signal.
df2.head()
df2.sum(axis=0)

2021-09-15 16:30:00+01:00    0.0
2021-09-15 17:00:00+01:00    0.0
2021-09-15 17:30:00+01:00    1.0
2021-09-15 18:00:00+01:00    2.0
2021-09-15 18:30:00+01:00    3.0
                            ... 
2021-11-11 20:30:00+00:00    2.0
2021-11-11 21:00:00+00:00    2.0
2021-11-11 21:30:00+00:00    2.0
2021-11-11 22:00:00+00:00    2.0
2021-11-11 22:30:00+00:00    2.0
Length: 1942, dtype: float64

## Then we can put everything into a function.

In [29]:

def gettriggers(df, lags, buy=True):
    df2 = pd.DataFrame()
    for i in range(1, lags+1):
        if buy:
            mask= (df['%K'].shift(i) < 20) & (df['%D'].shift(i) < 20)
        else: 
            mask= (df['%K'].shift(i) > 80) & (df['%D'].shift(i) > 80)
        df2 = df2.append(mask,ignore_index=True)
    return df2.sum(axis=0)


In [30]:
df['Buytrigger'] = np.where(gettriggers(df, 4),1,0) # if we get a buy signal (sum is larger than 0) we get a 1, if we dont we get a 0.

In [31]:
df['Selltrigger'] = np.where(gettriggers(df,4, False),1,0)

In [38]:
df['Buy'] = np.where((df.Buytrigger) & (df["%K"].between(20,80)) & (df["%D"].between(20,80)) & (df.rsi>50) & (df.macd > 0),1,0)

In [39]:
df['Sell'] = np.where((df.Selltrigger) & (df['%K'].between(20,80)) & (df['%D'].between(20,80)) & (df.rsi >50) & (df.macd > 0),1,0)

In [40]:
Buying_dates, Selling_dates = [], []

In [41]:
for i in range(len(df) - 1): 
    if df.Buy.iloc[i]:
        Buying_dates.append(df.iloc[i+1].name)
        for num,j in enumerate(df.Sell[i:]):
            if j:
                Selling_dates.append(df.iloc[i + num + 1].name)
                break

In [42]:
cutit =len(Buying_dates) - len(Selling_dates)

In [45]:
if cutit:
    Buying_dates = Buying_dates[:-cutit]

In [46]:
frame = pd.DataFrame({'Buying_dates':Buying_dates, 'Selling_dates': Selling_dates})

In [47]:
frame

,Buying_dates,Selling_dates
0,2021-09-22 03:00:00+01:00,2021-09-23 09:00:00+01:00
1,2021-09-28 13:00:00+01:00,2021-10-01 10:00:00+01:00
2,2021-10-07 08:00:00+01:00,2021-10-08 12:00:00+01:00
3,2021-10-07 08:30:00+01:00,2021-10-08 12:00:00+01:00
4,2021-10-14 21:00:00+01:00,2021-10-15 05:30:00+01:00
5,2021-10-20 03:00:00+01:00,2021-10-20 20:30:00+01:00
6,2021-10-27 13:00:00+01:00,2021-10-27 15:00:00+01:00
7,2021-10-28 04:30:00+01:00,2021-11-01 10:30:00+00:00
8,2021-11-05 05:30:00+00:00,2021-11-05 18:30:00+00:00


In [48]:
actuals = frame[frame.Buying_dates > frame.Selling_dates.shift(1)]

In [49]:
actuals

,Buying_dates,Selling_dates
1,2021-09-28 13:00:00+01:00,2021-10-01 10:00:00+01:00
2,2021-10-07 08:00:00+01:00,2021-10-08 12:00:00+01:00
4,2021-10-14 21:00:00+01:00,2021-10-15 05:30:00+01:00
5,2021-10-20 03:00:00+01:00,2021-10-20 20:30:00+01:00
6,2021-10-27 13:00:00+01:00,2021-10-27 15:00:00+01:00
7,2021-10-28 04:30:00+01:00,2021-11-01 10:30:00+00:00
8,2021-11-05 05:30:00+00:00,2021-11-05 18:30:00+00:00
